In [1]:
print(f"ipython: {str(get_ipython())}")
from IPython.display import clear_output
if 'google.colab' in str(get_ipython()):
    import sys
    from google.colab import drive, output
    drive.mount('/content/drive')
    !pip install torchaudio
    !pip install wandb --upgrade
    !wandb login
    # !pip install umap-learn
    output.clear()
    print("Running on colab")
    %load_ext autoreload
    %autoreload 1
    %cd '/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project'
    #sys.path.append('/content/drive/MyDrive/Deep Learning/AutoEncoders/Project/VQVAE_Working/data')
    #sys.path.append('/content/drive/MyDrive/Deep Learning/AutoEncoders/Project/VQVAE_Working/models')
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Dataloader')
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Models')
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Modules')
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Clustering_Results/Results')
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Clustering_Results/Figures')
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Clustering_Result')
    root_path = 'ConservacionBiologicaIA/Datos/Jaguas_2018'
elif "zmqshell" in str(get_ipython()):
    print("Running on MIRP")
    root_path = 'media/mirp_ai/Seagate Desktop Drive/Jaguas_2018'
else:
    print("Running in personal pc")
    root_path = 'ConservacionBiologicaIA/Datos/Jaguas_2018'


ipython: <ipykernel.zmqshell.ZMQInteractiveShell object at 0x7ff48e7f9ba0>
Running on MIRP


In [2]:
import numpy as np
import torch 
import torchvision 
import torchaudio
import torch.nn as nn
import math
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import random_split
from Jaguas_DataLoader_rainless import SoundscapeData
from PosAE_training_functions import posautoencoding_m1,TestModel, TrainModel
!pip install positional-encodings[pytorch]
from positional_encodings.torch_encodings import PositionalEncoding1D, PositionalEncoding2D, PositionalEncoding3D, Summer, PositionalEncodingPermute2D
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
!pip install wandb --upgrade
!wandb login
import wandb
from IPython.display import clear_output
import datetime
wandb.login()

import random
def _set_seed(seed: int):
    """
    Helper function for reproducible behavior to set the seed in ``random``, ``numpy``, ``torch`` and/or ``tf`` (if
    installed).
 
    Args:
        seed (:obj:`int`): The seed to set.
    """
    random.seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        # ^^ safe to call this function even if cuda is not available
_set_seed(1024)

wandb: Currently logged in as: danielnieto. Use `wandb login --relogin` to force relogin


wandb: Currently logged in as: danielnieto. Use `wandb login --relogin` to force relogin


class PositionalEncoding2d(nn.Module):

    def __init__(self, d_model: int=64, height: int = 9, width: int =9, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(d_model, height, width)
        # Each dimension use half of d_model
        d_model = int(d_model / 2)
        div_term = torch.exp(torch.arange(0., d_model, 2) *
                             -(math.log(10000.0) / d_model))
        pos_w = torch.arange(0., width).unsqueeze(1)
        pos_h = torch.arange(0., height).unsqueeze(1)
        pe[0:d_model:2, :, :] = torch.sin(pos_w * div_term).transpose(0, 1).unsqueeze(1).repeat(1, height, 1)
        pe[1:d_model:2, :, :] = torch.cos(pos_w * div_term).transpose(0, 1).unsqueeze(1).repeat(1, height, 1)
        pe[d_model::2, :, :] = torch.sin(pos_h * div_term).transpose(0, 1).unsqueeze(2).repeat(1, 1, width)
        pe[d_model + 1::2, :, :] = torch.cos(pos_h * div_term).transpose(0, 1).unsqueeze(2).repeat(1, 1, width)
        self.register_buffer('pe', pe)
        
    def forward(self, x, index: int, dropout: bool=False):
        """
        Args:
            x: Tensor, shape [seq_len, batch_size, embedding_dim]
        """
        x = x.to("cuda")
        self.pe = self.pe.to("cuda")
#         print(x.shape, self.pe.shape)
        x = x + self.pe[index]
        if dropout:
            x = self.dropout(x)
        else:
            x = x
        return x

class posautoencoding_m1(nn.Module):

    """
    Convolutional autoencoder made to reconstruct the audios spectrograms generated by the EcoDataTesis dataloader.
    """

    def __init__(self, num_hiddens: int = 64):
        """
        Constructor of the convolutional autoencoder model.
        """
        super().__init__()
        # TODO: To design the final architechture considering the spectrograms sizes.
        # TODO: To correct the current sizes of the decoder.

        self.encoder = nn.Sequential(
            nn.Conv2d(1, num_hiddens // 8, kernel_size=8, stride=3, padding=0),  # N, 256, 127, 8004
            nn.ReLU(),
            nn.Conv2d(num_hiddens // 8, num_hiddens // 4, kernel_size=8, stride=3, padding=0),  # N, 512, 125,969
            nn.ReLU(),
            nn.Conv2d(num_hiddens // 4, num_hiddens // 2, kernel_size=4, stride=3, padding=0),  # N, 512, 125,969
            nn.ReLU(),
            nn.Conv2d(num_hiddens // 2, num_hiddens, kernel_size=2, stride=2, padding=0),  # N, 512, 125,969
            nn.ReLU()
             )
        self.decoder = nn.Sequential(  # This is like go in opposite direction respect the encoder
            nn.ConvTranspose2d(num_hiddens, num_hiddens // 2, kernel_size=2, stride=2, padding=0, output_padding=0),  # N, 32, 126,8000
            nn.ReLU(),
            nn.ConvTranspose2d(num_hiddens // 2, num_hiddens // 4, kernel_size=4, stride=3, padding=0, output_padding=0),  # N, 32, 127,64248
            nn.ReLU(),
            nn.ConvTranspose2d(num_hiddens // 4, num_hiddens // 8, kernel_size=8, stride=3, padding=0, output_padding=0),  # N, 32, 127,64248
            nn.ReLU(),
            nn.ConvTranspose2d(num_hiddens // 8, 1, kernel_size=8, stride=3, padding=0, output_padding=0),  # N, 32, 127,64248
            nn.Sigmoid()

            )
            
            
    def forward(self, x, y, max_len=24):
        
        """
        Method to compute an image output based on the performed model.

        :param x: Input spectrogram images as tensors.
        :type x: torch.tensor
        :return: Reconstructed images
        """
        
        #print(f"x_shape:{x.shape}")
        encoded = self.encoder(x)
#         print("encoded: ", encoded.shape)
        pos_encoder = PositionalEncoding2d(64, dropout = 0.1, max_len = 24).to("cuda")
        posencoding_2d = pos_encoder(encoded.permute(1,0,2,3), y)
#         print(posencoding_2d)
        posencoding_2d = posencoding_2d.permute(1,0,2,3)
#         print("encoder_shape: ", encoded.shape)
        decoded = self.decoder(posencoding_2d)
#         print("decoder_shape: ",decoded.shape)
        return decoded

In [3]:
filters = {"Intensity_Category": "No_rain"}
dataset = SoundscapeData(root_path, dataframe_path="Complementary_Files/Audios_Jaguas/Audios_Jaguas.csv",
                         audio_length=12, ext="wav",
                         win_length=1028, filters=filters)
dataset_train, dataset_test = random_split(dataset,
                                           [round(len(dataset)*0.8), len(dataset) - round(len(dataset)*0.8)], 
                                           generator=torch.Generator().manual_seed(1024))

config = {
    "project" : "positionalAE-Jaguas_Hour",
    "audio_length": dataset.audio_length,
    "batch_size" : 14,
    "num_epochs": 5,
    "num_hiddens" : 64,
    "gamma_lr" : 0.1,
    "learning_rate" : 1e-3,
    "dataset" : "Audios Jaguas",
    "architecture": "PositionalAE_Recorder",
    "win_length" : dataset.win_length,
    "step_size": 5,
}

training_loader = DataLoader(dataset_train, batch_size=config["batch_size"])
test_loader = DataLoader(dataset_test, batch_size=config["batch_size"])

model = posautoencoding_m1(num_hiddens=config["num_hiddens"]).to("cuda")

optimizer = optim.Adam(model.parameters(), lr=config["learning_rate"], amsgrad=False)
scheduler = lr_scheduler.StepLR(optimizer, step_size = config["step_size"], gamma = config["gamma_lr"] )

config["optimizer"] = optimizer
config["scheduler"] = scheduler
config["num_training_updates"] = len(training_loader)

/media/mirp_ai/Seagate Desktop Drive/Jaguas_2018
Running PosEncoding_m1


In [4]:
Training = TrainModel(model=model.to("cuda"))
model, logs, run_name = Training.forward(training_loader, test_loader, config)
time = datetime.datetime.now()
torch.save(model.state_dict(),f'temporal/models/model_{run_name}_month_{time.month}_day_{time.day}_hour_{time.hour}_final.pth')
torch.save(config,f'temporal/configs/config_{run_name}_day_{time.day}_hour_{time.hour}.pth')
torch.save(dataset_test, f"temporal/datasets/dataset_test_posae_jaguas_{time.day}_70%.pth")
torch.save(dataset_train, f"temporal/datasets/dataset_train_posae_jaguas_{time.day}_70%.pth")

0.0001


loss,▄▆▂▃▃▂▅▃▂▄▅▃▂▁▃▃▃▂▂▄▂▁▁▅▂▁▆▂▅▅▁▁▂▂█▂▃▁▂▁
step,▁▃▆█▁▃▆█▁▃▆█▁▃▆█▁▃▆█
loss,0.1011
step,4


In [5]:
iterador = iter(training_loader)
data, b, c, d = next(iterador)
data = torch.reshape(data, (data.shape[0] * data.shape[1] *data.shape[2], data.shape[3], data.shape[4]))
data = torch.unsqueeze(data, 1)
data.shape

torch.Size([70, 1, 515, 515])

In [6]:
training_loader = DataLoader(dataset_train, batch_size=1)
test_loader = DataLoader(dataset_test, batch_size=2)
iterator = iter(test_loader)
testing = TestModel(model, iterator, device=torch.device("cuda"))

In [7]:
originals, reconstructions, encodings, loss, label, path = testing.reconstruct()

In [8]:
encodings.shape

torch.Size([10, 64, 9, 9])

# Featurer

In [9]:
import os
os.listdir()

['Models',
 'vggish',
 'Projection_PosAE__UMAP_recorder.png',
 'Jaguas_Dataframes.ipynb',
 'temporal',
 'Projection_PosAE__Scaled_recorder.png',
 'PositionalAE_Recorder_batch_size_14_num_hiddens_64__month_7_day_4_recorder_14_epoch_1.pkl',
 'AE_Jaguas.ipynb',
 'VAE_Jaguas.ipynb',
 'Encodec',
 'Pythae',
 'wandb',
 'VQ_VAE_training_functions.py',
 'PositionalAE_Recorder_batch_size_14_num_hiddens_64__month_7_day_4_recorder_15_epoch_2.pkl',
 'CGPT_Positional_Encodings.ipynb',
 'PositionalAE_Recorder_batch_size_14_num_hiddens_64__month_7_day_4_recorder_18_epoch_5.pkl',
 'Projection_PosAE__PCA60_recorder.png',
 'AE_Clustering_Jaguas.ipynb',
 'PositionalAE_Recorder_batch_size_14_num_hiddens_64__month_7_day_4_recorder_16_epoch_3.pkl',
 'Complementary_Files',
 'Projection_Images',
 'PositionalAE_Recorder_batch_size_14_num_hiddens_64__month_7_day_4_recorder_17_epoch_4.pkl',
 'Spatial_Clustering.ipynb',
 'acoustic_exploration',
 'Positional_Encodings.ipynb',
 'AE_Jaguas_HPSS.ipynb',
 'Jaguas_Data_

In [10]:
model_name = f"temporal/models/model_PositionalAE_hour_batch_size_14_num_hiddens_64__day_10_hour_21_final.pth"
config = torch.load(f'temporal/configs/config_PositionalAE_hour_batch_size_14_num_hiddens_64__day_10_hour_21.pth', map_location=torch.device('cpu'))
model = posautoencoding_m1(num_hiddens=config["num_hiddens"]).to("cuda")
dataset_test = torch.load(f'temporal/datasets/dataset_test_posae_jaguas_22_70%.pth')
dataset_train = torch.load(f'temporal/datasets/dataset_train_posae_jaguas_22_70%.pth')
model.load_state_dict(torch.load(f'{model_name}', map_location=torch.device('cpu')))

Running PosEncoding_m1


<All keys matched successfully>

In [11]:
from six.moves import xrange
training_loader = DataLoader(dataset_train, batch_size=1)
test_loader = DataLoader(dataset_test, batch_size=1)
iterator = iter(test_loader)
testing = TestModel(model, iterator, device=torch.device("cuda"))
encodings_size = encodings[0].shape

training_recorder_list = []
training_hour_list = []
training_minute_list = []
delete_samples = []
training_path_samples = []
training_samples_list_torch = []
for id in range(len(test_loader)):
#     if (id+1)%5 == 0:
#         print("finished")
#         break
    if (id+1)% 500 == 0:
        print(f"id: {id + 1} of {len(dataset_test)}")
    try:
        originals, reconstructions, encodings, label, loss, path = testing.reconstruct()
    except:
        print(f"error id: {id}")
        delete_samples.append(id)
        continue

    encodings_size = encodings[0].shape
    encodings = encodings.to("cuda").detach()
    encodings = encodings.reshape(encodings.shape[0],
                                encodings.shape[1]*encodings.shape[2]*encodings.shape[3])
    encoding = encodings.squeeze(dim=0)
    training_samples_list_torch.append(encodings)
    training_recorder_list.append(label["recorder"].reshape(label["recorder"].shape[0]*label["recorder"].shape[1]))
    training_hour_list.append(label["hour"].reshape(label["hour"].shape[0]*label["hour"].shape[1]))
    training_minute_list.append(label["minute"].reshape(label["minute"].shape[0]*label["minute"].shape[1]))
    
    
    path = np.asarray(path)
    path = np.repeat(path, 5)
    training_path_samples.append(path)

training_recorder_list = torch.cat(training_recorder_list,dim=0)
training_hour_list = torch.cat(training_hour_list,dim=0)
training_minute_list = torch.cat(training_minute_list,dim=0)
training_samples_list_torch = torch.cat(training_samples_list_torch, dim=0)

torch.save(training_path_samples, "Features/test_path_samples_posae_hour.pth")
torch.save(training_samples_list_torch, "Features/test_samples_list_torch_70%_posae_hour.pth")
torch.save(training_recorder_list, "Features/test_recorder_list_70%_posae_hour.pth")
torch.save(training_hour_list, "Features/test_hour_list_70%_posae_hour.pth")
torch.save(training_minute_list, "Features/test_minute_list_70%_posae_hour.pth")
training_labels_list = {"recorder": training_recorder_list, "hour": training_hour_list, "minute": training_minute_list}
torch.save(training_labels_list, "Features/test_labels_list_70%_posae_hour.pth")
torch.save(delete_samples, "Features/test_corrupted_samples_list_70%_posae_hour.pth")

error id: 0
error id: 1
error id: 2
error id: 3
error id: 4
error id: 5
error id: 6
error id: 7
error id: 8
error id: 9
error id: 10
error id: 11
error id: 12
error id: 13
error id: 14
error id: 15
error id: 16
error id: 17
error id: 18
error id: 19
error id: 20
error id: 21
error id: 22
error id: 23
error id: 24
error id: 25
error id: 26
error id: 27
error id: 28
error id: 29
error id: 30
error id: 31
error id: 32
error id: 33
error id: 34
error id: 35
error id: 36
error id: 37
error id: 38
error id: 39
error id: 40
error id: 41
error id: 42
error id: 43
error id: 44
error id: 45
error id: 46
error id: 47
error id: 48
error id: 49
error id: 50
error id: 51
error id: 52
error id: 53
error id: 54
error id: 55
error id: 56
error id: 57
error id: 58
error id: 59
error id: 60
error id: 61
error id: 62
error id: 63
error id: 64
error id: 65
error id: 66
error id: 67
error id: 68
error id: 69
error id: 70
error id: 71
error id: 72
error id: 73
error id: 74
error id: 75
error id: 76
error id:

error id: 643
error id: 644
error id: 645
error id: 646
error id: 647
error id: 648
error id: 649
error id: 650
error id: 651
error id: 652
error id: 653
error id: 654
error id: 655
error id: 656
error id: 657
error id: 658
error id: 659
error id: 660
error id: 661
error id: 662
error id: 663
error id: 664
error id: 665
error id: 666
error id: 667
error id: 668
error id: 669
error id: 670
error id: 671
error id: 672
error id: 673
error id: 674
error id: 675
error id: 676
error id: 677
error id: 678
error id: 679
error id: 680
error id: 681
error id: 682
error id: 683
error id: 684
error id: 685
error id: 686
error id: 687
error id: 688
error id: 689
error id: 690
error id: 691
error id: 692
error id: 693
error id: 694
error id: 695
error id: 696
error id: 697
error id: 698
error id: 699
error id: 700
error id: 701
error id: 702
error id: 703
error id: 704
error id: 705
error id: 706
error id: 707
error id: 708
error id: 709
error id: 710
error id: 711
error id: 712
error id: 713
error 

error id: 1291
error id: 1292
error id: 1293
error id: 1294
error id: 1295
error id: 1296
error id: 1297
error id: 1298
error id: 1299
error id: 1300
error id: 1301
error id: 1302
error id: 1303
error id: 1304
error id: 1305
error id: 1306
error id: 1307
error id: 1308
error id: 1309
error id: 1310
error id: 1311
error id: 1312
error id: 1313
error id: 1314
error id: 1315
error id: 1316
error id: 1317
error id: 1318
error id: 1319
error id: 1320
error id: 1321
error id: 1322
error id: 1323
error id: 1324
error id: 1325
error id: 1326
error id: 1327
error id: 1328
error id: 1329
error id: 1330
error id: 1331
error id: 1332
error id: 1333
error id: 1334
error id: 1335
error id: 1336
error id: 1337
error id: 1338
error id: 1339
error id: 1340
error id: 1341
error id: 1342
error id: 1343
error id: 1344
error id: 1345
error id: 1346
error id: 1347
error id: 1348
error id: 1349
error id: 1350
error id: 1351
error id: 1352
error id: 1353
error id: 1354
error id: 1355
error id: 1356
error id: 

error id: 1938
error id: 1939
error id: 1940
error id: 1941
error id: 1942
error id: 1943
error id: 1944
error id: 1945
error id: 1946
error id: 1947
error id: 1948
error id: 1949
error id: 1950
error id: 1951
error id: 1952
error id: 1953
error id: 1954
error id: 1955
error id: 1956
error id: 1957
error id: 1958
error id: 1959
error id: 1960
error id: 1961
error id: 1962
error id: 1963
error id: 1964
error id: 1965
error id: 1966
error id: 1967
error id: 1968
error id: 1969
error id: 1970
error id: 1971
error id: 1972
error id: 1973
error id: 1974
error id: 1975
error id: 1976
error id: 1977
error id: 1978
error id: 1979
error id: 1980
error id: 1981
error id: 1982
error id: 1983
error id: 1984
error id: 1985
error id: 1986
error id: 1987
error id: 1988
error id: 1989
error id: 1990
error id: 1991
error id: 1992
error id: 1993
error id: 1994
error id: 1995
error id: 1996
error id: 1997
error id: 1998
id: 2000 of 6020
error id: 1999
error id: 2000
error id: 2001
error id: 2002
error id

error id: 2582
error id: 2583
error id: 2584
error id: 2585
error id: 2586
error id: 2587
error id: 2588
error id: 2589
error id: 2590
error id: 2591
error id: 2592
error id: 2593
error id: 2594
error id: 2595
error id: 2596
error id: 2597
error id: 2598
error id: 2599
error id: 2600
error id: 2601
error id: 2602
error id: 2603
error id: 2604
error id: 2605
error id: 2606
error id: 2607
error id: 2608
error id: 2609
error id: 2610
error id: 2611
error id: 2612
error id: 2613
error id: 2614
error id: 2615
error id: 2616
error id: 2617
error id: 2618
error id: 2619
error id: 2620
error id: 2621
error id: 2622
error id: 2623
error id: 2624
error id: 2625
error id: 2626
error id: 2627
error id: 2628
error id: 2629
error id: 2630
error id: 2631
error id: 2632
error id: 2633
error id: 2634
error id: 2635
error id: 2636
error id: 2637
error id: 2638
error id: 2639
error id: 2640
error id: 2641
error id: 2642
error id: 2643
error id: 2644
error id: 2645
error id: 2646
error id: 2647
error id: 

...

error id: 3869
error id: 3870
error id: 3871
error id: 3872
error id: 3873
error id: 3874
error id: 3875
error id: 3876
error id: 3877
error id: 3878
error id: 3879
error id: 3880
error id: 3881
error id: 3882
error id: 3883
error id: 3884
error id: 3885
error id: 3886
error id: 3887
error id: 3888
error id: 3889
error id: 3890
error id: 3891
error id: 3892
error id: 3893
error id: 3894
error id: 3895
error id: 3896
error id: 3897
error id: 3898
error id: 3899
error id: 3900
error id: 3901
error id: 3902
error id: 3903
error id: 3904
error id: 3905
error id: 3906
error id: 3907
error id: 3908
error id: 3909
error id: 3910
error id: 3911
error id: 3912
error id: 3913
error id: 3914
error id: 3915
error id: 3916
error id: 3917
error id: 3918
error id: 3919
error id: 3920
error id: 3921
error id: 3922
error id: 3923
error id: 3924
error id: 3925
error id: 3926
error id: 3927
error id: 3928
error id: 3929
error id: 3930
error id: 3931
error id: 3932
error id: 3933
error id: 3934
error id: 

error id: 4511
error id: 4512
error id: 4513
error id: 4514
error id: 4515
error id: 4516
error id: 4517
error id: 4518
error id: 4519
error id: 4520
error id: 4521
error id: 4522
error id: 4523
error id: 4524
error id: 4525
error id: 4526
error id: 4527
error id: 4528
error id: 4529
error id: 4530
error id: 4531
error id: 4532
error id: 4533
error id: 4534
error id: 4535
error id: 4536
error id: 4537
error id: 4538
error id: 4539
error id: 4540
error id: 4541
error id: 4542
error id: 4543
error id: 4544
error id: 4545
error id: 4546
error id: 4547
error id: 4548
error id: 4549
error id: 4550
error id: 4551
error id: 4552
error id: 4553
error id: 4554
error id: 4555
error id: 4556
error id: 4557
error id: 4558
error id: 4559
error id: 4560
error id: 4561
error id: 4562
error id: 4563
error id: 4564
error id: 4565
error id: 4566
error id: 4567
error id: 4568
error id: 4569
error id: 4570
error id: 4571
error id: 4572
error id: 4573
error id: 4574
error id: 4575
error id: 4576
error id: 

error id: 5152
error id: 5153
error id: 5154
error id: 5155
error id: 5156
error id: 5157
error id: 5158
error id: 5159
error id: 5160
error id: 5161
error id: 5162
error id: 5163
error id: 5164
error id: 5165
error id: 5166
error id: 5167
error id: 5168
error id: 5169
error id: 5170
error id: 5171
error id: 5172
error id: 5173
error id: 5174
error id: 5175
error id: 5176
error id: 5177
error id: 5178
error id: 5179
error id: 5180
error id: 5181
error id: 5182
error id: 5183
error id: 5184
error id: 5185
error id: 5186
error id: 5187
error id: 5188
error id: 5189
error id: 5190
error id: 5191
error id: 5192
error id: 5193
error id: 5194
error id: 5195
error id: 5196
error id: 5197
error id: 5198
error id: 5199
error id: 5200
error id: 5201
error id: 5202
error id: 5203
error id: 5204
error id: 5205
error id: 5206
error id: 5207
error id: 5208
error id: 5209
error id: 5210
error id: 5211
error id: 5212
error id: 5213
error id: 5214
error id: 5215
error id: 5216
error id: 5217
error id: 

error id: 5795
error id: 5796
error id: 5797
error id: 5798
error id: 5799
error id: 5800
error id: 5801
error id: 5802
error id: 5803
error id: 5804
error id: 5805
error id: 5806
error id: 5807
error id: 5808
error id: 5809
error id: 5810
error id: 5811
error id: 5812
error id: 5813
error id: 5814
error id: 5815
error id: 5816
error id: 5817
error id: 5818
error id: 5819
error id: 5820
error id: 5821
error id: 5822
error id: 5823
error id: 5824
error id: 5825
error id: 5826
error id: 5827
error id: 5828
error id: 5829
error id: 5830
error id: 5831
error id: 5832
error id: 5833
error id: 5834
error id: 5835
error id: 5836
error id: 5837
error id: 5838
error id: 5839
error id: 5840
error id: 5841
error id: 5842
error id: 5843
error id: 5844
error id: 5845
error id: 5846
error id: 5847
error id: 5848
error id: 5849
error id: 5850
error id: 5851
error id: 5852
error id: 5853
error id: 5854
error id: 5855
error id: 5856
error id: 5857
error id: 5858
error id: 5859
error id: 5860
error id: 

RuntimeError: torch.cat(): expected a non-empty list of Tensors

In [ ]:
test = TestModel(model, iterador)

In [ ]:
posenc = PositionalEncoding2d() 
x = posenc(encodes.permute(1,0,2,3), c["recorder"].reshape(14*5))

In [ ]:
posenc = PositionalEncoding2d() 
encodes = model.encoder(data.to("cuda"))
print(encodes.shape)
posencoding_2d = posenc(data, c["recorder"].reshape(14*5))
decoded = model.decoder(posencoding_2d)
